In [1]:
import sys
sys.path.append("..")
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pickle
import random
import numpy as np
import copy
import json
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [2]:
import wandb
wandb.init(mode="disabled")

In [3]:
from das.Classification_Model import (MLPForClassification,
                                  train_model,
                                  eval_model,
                                  test_model,
                                  make_model)
from das.Helper_Functions import set_seed
from das.Dataset_Generation import (make_model_dataset,
                                make_model_dataset_AndOrAnd,
                                make_intervention_dataset_variable_intervention_all,
                                make_intervention_dataset_variable_intervention_first,
                                make_intervention_dataset_first_input_intervention,
                                make_intervention_dataset_AndOrAnd,
                                make_intervention_dataset_AndOr)
from das.RevNet import RevNet
from das.Rotation_Model import Rotation
from das.DAS import phi_class
from das.DAS_MLP import Distributed_Alignment_Search_MLP

In [4]:
DEVICE  = "cpu" #"cuda"/"cpu"
num_classes=2

In [5]:
def register_intervention_hook(Save_array,Pos,layer):
    def hook_fn(module, input, output):
        Save_array[Pos].append(output.detach().cpu())
    layer.register_forward_hook(hook_fn)

In [6]:
Full_results=[]
for acseed in [4287, 3837, 9097, 2635, 5137, 6442, 5234, 4641, 8039, 2266]:
    set_seed(acseed)
    X_train,y_train = make_model_dataset(1048576,4,DEVICE)#1048576
    X_eval,y_eval   = make_model_dataset(10000,4,DEVICE)#10000
    X_test,y_test   = make_model_dataset(10000,4,DEVICE)
    X_inj,y_inj   = make_model_dataset(10000,4,DEVICE)#5000
   
    model,accuracy=make_model(X_train,y_train,X_eval,y_eval,X_test,y_test,input_size=16,epochs=20,device=DEVICE)
    Layers=[]
    Layers.append(("Layer1",model.mlp.h[0]))
    Layers.append(("Layer2",model.mlp.h[1]))
    Layers.append(("Layer3",model.mlp.h[2]))


    Layer_Save=[[],[],[],[]]
    Results=[]
    Results.append([0,0,0,0,0,0,0])
    Hooks=[]
    predicted_classes_set = set()
    for acpos,aclayer in enumerate(Layers):
        Layer_Save.append([])
        Results.append([0,0,0,0,0,0,0])
        Hooks.append(register_intervention_hook(Layer_Save,acpos+1,aclayer[1]))
    
    
    test_dataset = TensorDataset(X_inj,y_inj)
    test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)#6400
    
    
    model.eval()
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            Layer_Save[0].append(X_batch)
            output=model(X_batch)
            predicted = torch.argmax(output, dim=1)
            Layer_Save[-3].append(predicted)
            Layer_Save[-2].append(torch.all(X_batch[:, :4] == X_batch[:, 4:8], dim=1))
            Layer_Save[-1].append(torch.all(X_batch[:, 8:12] == X_batch[:, 12:16], dim=1))
        
            # Update set with predictions from this batch
            predicted_classes_set.update(predicted.tolist())
    for i in range(len(Layer_Save)):
        Layer_Save[i]=torch.cat(Layer_Save[i])
    for i in tqdm(range(len(Layer_Save[0]))):
        for j in range(i+1,len(Layer_Save[0])):
            Results[0][0]+=1
            Results[0][1]+=torch.norm(Layer_Save[0][i]-Layer_Save[0][j], p=2).item()
            if Layer_Save[-3][i]==Layer_Save[-3][j]:
                Results[0][3]+=1
                Results[0][4]+=torch.norm(Layer_Save[0][i]-Layer_Save[0][j], p=2).item()
            if Layer_Save[-2][i]==Layer_Save[-2][j] and Layer_Save[-1][i]==Layer_Save[-1][j]:
                Results[0][5]+=1
                Results[0][6]+=torch.norm(Layer_Save[0][i]-Layer_Save[0][j], p=2).item()
            if torch.equal(Layer_Save[0][i],Layer_Save[0][j]):
                Results[0][2]+=1
            for k in range(1,len(Layer_Save)-3):
                Results[k][0]+=1
                Results[k][1]+=torch.norm(Layer_Save[k][i]-Layer_Save[k][j], p=2).item()
                if torch.equal(Layer_Save[k][i],Layer_Save[k][j]):
                    Results[k][2]+=1
                if Layer_Save[-3][i]==Layer_Save[-3][j]:
                    Results[k][3]+=1
                    Results[k][4]+=torch.norm(Layer_Save[k][i]-Layer_Save[k][j], p=2).item()
                if Layer_Save[-2][i]==Layer_Save[-2][j] and Layer_Save[-1][i]==Layer_Save[-1][j]:
                    Results[k][5]+=1
                    Results[k][6]+=torch.norm(Layer_Save[k][i]-Layer_Save[k][j], p=2).item()
    
    all_classes_set = set(range(num_classes))
    missing_classes_set = all_classes_set - predicted_classes_set
    Results_processed=[]
    for i in Results:
        Results_processed.append([i[1]/i[0],i[2]/i[0],i[4]/i[3],i[6]/i[5]])
    print("Surjectivity: Missing:", missing_classes_set,"Found:",predicted_classes_set)
    print("Injectivity", Results_processed)  
    print("Injectivity", Results) 
    Full_results.append({})
    Full_results[-1]["Results"]=Results
    Full_results[-1]["Results_processed"]=Results_processed
    Full_results[-1]["missing_classes_set"]=list(missing_classes_set)
    Full_results[-1]["predicted_classes_set"]=list(predicted_classes_set)
    with open('results.json', 'w') as f:
        json.dump(Full_results, f)

/home/master/DAS/Complexity-vs.-Accuracy/notebooks/../das/Dataset_Generation.py:68: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  return torch.tensor(model_inputs, dtype=torch.float32).to(device),torch.tensor(labels, dtype=torch.float32).to(device)


Epoch 1, Loss: 0.39049801870714873 steps without improvement: 7 best accuracy: 0.9837
Epoch 2, Loss: 0.026408711896237946 steps without improvement: 72 best accuracy: 0.9964
Epoch 3, Loss: 0.008641325565577063 steps without improvement: 37 best accuracy: 0.9986
Epoch 4, Loss: 0.004503298025667846 steps without improvement: 60 best accuracy: 0.9991
Epoch 5, Loss: 0.0028202552880998155 steps without improvement: 138 best accuracy: 0.9997
Epoch 6, Loss: 0.0019203961838059058 steps without improvement: 301 best accuracy: 0.9998


100%|███████████████████████████████████| 10000/10000 [3:01:04<00:00,  1.09s/it]


Surjectivity: Missing: set() Found: {0, 1}
Injectivity [[1.6062524000963159, 0.0, 1.6062073364124536, 1.6027513844150898], [1.1187233052468357, 0.0, 1.0942841669223096, 0.9170955777466178], [2.0019155553167427, 0.0, 1.905394463454935, 1.2899510248964001], [4.199196621574812, 0.0, 3.22916042301307, 2.124650866420493]]
Injectivity [[49995000, 80304588.74281532, 0, 24995900, 40148597.96023205, 12499010, 20032805.58131805], [49995000, 55930571.64581555, 0, 24995900, 27352617.607973356, 12499010, 11462786.797210753], [49995000, 100085768.18806055, 0, 24995900, 47627049.469073206, 12499010, 16123110.759690354], [49995000, 209938835.0956327, 0, 24995900, 80715771.0175924, 12499010, 26556032.425898407]]
Epoch 1, Loss: 0.4398490324092563 steps without improvement: 4 best accuracy: 0.9509
Epoch 2, Loss: 0.04545272669111 steps without improvement: 24 best accuracy: 0.9961
Epoch 3, Loss: 0.008210613968117286 steps without improvement: 0 best accuracy: 0.9986
Epoch 4, Loss: 0.003960001421773995 ste

100%|███████████████████████████████████| 10000/10000 [3:04:28<00:00,  1.11s/it]


...

Surjectivity: Missing: set() Found: {0, 1}
Injectivity [[1.6061571386875442, 0.0, 1.6061696141760247, 1.6028435674240267], [1.2241925686496185, 0.0, 1.2020133305599345, 1.0290868504151802], [2.721948233475916, 0.0, 2.6377805771929097, 2.0457959256244997], [4.927898843784046, 0.0, 4.534750237042202, 3.3554525440732865]]
Injectivity [[49995000, 80299826.14868377, 0, 24997304, 40149910.121120796, 12497797, 20032013.528421298], [49995000, 61203507.46963767, 0, 24997304, 30047092.636059172, 12497797, 12861318.551858287], [49995000, 136083801.93262842, 0, 24997304, 65937402.97338663, 12497797, 25567942.1818821], [49995000, 246370302.69498336, 0, 24997304, 113356530.23941597, 12497797, 41935764.73896149]]
Epoch 1, Loss: 0.4044290292367805 steps without improvement: 0 best accuracy: 0.9618
Epoch 2, Loss: 0.048057374328891456 steps without improvement: 33 best accuracy: 0.9949
Epoch 3, Loss: 0.011773959729907801 steps without improvement: 81 best accuracy: 0.9981
Epoch 4, Loss: 0.00615440726539

100%|███████████████████████████████████| 10000/10000 [3:04:57<00:00,  1.11s/it]


Surjectivity: Missing: set() Found: {0, 1}
Injectivity [[1.6043662035976995, 0.0, 1.60433194435454, 1.600887214108796], [1.0593074544599934, 0.0, 1.036685836156183, 0.8731391906782866], [1.2158356967631732, 0.0, 1.168732509342229, 0.8686391420728741], [3.0718428998910547, 0.0, 2.356094126087442, 1.6921212678935482]]
Injectivity [[49995000, 80210288.34886698, 0, 24995400, 40100918.68191947, 12496178, 20004971.585427627], [49995000, 52960076.18572737, 0, 24995400, 25912377.14905826, 12496178, 10910902.74549181], [49995000, 60785705.65967484, 0, 24995400, 29212936.564012747, 12496178, 10854669.337109923], [49995000, 153576785.7800533, 0, 24995400, 58891515.11920604, 12496178, 21145048.561183464]]
Epoch 1, Loss: 0.42178907026391244 steps without improvement: 7 best accuracy: 0.9588
Epoch 2, Loss: 0.054393591018197185 steps without improvement: 30 best accuracy: 0.9955
Epoch 3, Loss: 0.010968930734179594 steps without improvement: 116 best accuracy: 0.9989
Epoch 4, Loss: 0.00507179076180364

100%|███████████████████████████████████| 10000/10000 [3:05:08<00:00,  1.11s/it]

Surjectivity: Missing: set() Found: {0, 1}
Injectivity [[1.6074383553947849, 0.0, 1.6073831647598422, 1.6038395700585908], [0.8667285248388393, 0.0, 0.8474861772377078, 0.7255274033345864], [0.8236977808144543, 0.0, 0.7835643753990937, 0.5405897412703982], [2.5808393145957327, 0.0, 1.839171771947828, 1.1665662371696497]]
Injectivity [[49995000, 80363880.57796226, 0, 24995289, 40177006.73690687, 12495665, 20041041.98119618], [49995000, 43332092.599317774, 0, 24995289, 21183161.92356173, 12495665, 9065947.380388875], [49995000, 41180770.55181865, 0, 24995289, 19585418.01320484, 12495665, 6755028.30935157], [49995000, 129029061.53321366, 0, 24995289, 45970629.96047805, 12495665, 14577020.89998249]]


In [13]:
processed_results=[]
for ac_results in Full_results:
    for i in range(len(ac_results["Results_processed"])):
        if len(processed_results)-1<i:
            processed_results.append([])
        for j in range(len(ac_results["Results_processed"][i])):
            if len(processed_results[i])-1<j:
                processed_results[i].append([])
            processed_results[i][j].append(ac_results["Results_processed"][i][j])

In [14]:

for i in range(len(processed_results)):
    for j in range(len(processed_results[i])):
        processed_results[i][j]=np.array(processed_results[i][j])
        processed_results[i][j]=[round(np.mean(processed_results[i][j]),2),round(np.std(processed_results[i][j]),2)]

In [15]:
processed_results

[[[1.61, 0.0], [0.0, 0.0], [1.61, 0.0], [1.6, 0.0]],
 [[1.11, 0.15], [0.0, 0.0], [1.09, 0.14], [0.93, 0.13]],
 [[1.89, 0.72], [0.0, 0.0], [1.81, 0.71], [1.3, 0.64]],
 [[3.92, 1.16], [0.0, 0.0], [3.23, 1.24], [2.05, 0.95]]]